In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import time

from selenium.webdriver.chrome.options import Options

import time
from bs4 import BeautifulSoup
import pandas as pd
import requests

# options = Options()
# options.headless = True


# PATH = '/usr/local/bin/chromedriver'
PATH = 'INSERT DRIVER HERE'


# PATH = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(PATH)#, options=options)


driver.get('https://2022electionresults.comelec.gov.ph/#/er')



prec = ['77010001', '77070017']
df_container = []
for i in prec:
    search = driver.find_element_by_xpath('//*[@title="Search clustered precinct"]')
    search.click()
    time.sleep(3)
    active_ele = driver.switch_to.active_element
    active_ele.send_keys(i)
    active_ele.send_keys(Keys.RETURN)

    time.sleep(5)
    x = driver.page_source
    elementSource = driver.find_element_by_id("'resultDiv.'+5587").get_attribute("outerHTML")


    soup0 = BeautifulSoup(x, "lxml")
    soup = BeautifulSoup(elementSource, "lxml")
    link = soup.findAll("div", class_="text-left candidate-result sortable-val-1 ng-binding")
    loc = soup0.findAll("span", class_="region-name ng-binding ng-scope")

    loc = [' '.join(i.get_text().split()) for i in loc]

    results = soup.findAll("div", class_="candidate-result")

    xx = [' '.join(i.get_text().split()) for i in results]

    def split(a, n):
        k, m = divmod(len(a), n)
        return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

    df = pd.DataFrame(list(split(xx, 10)))
    dff = df[[0,1]].T
    dff = dff.rename(columns=dff.iloc[0])
    dff = dff.iloc[1:]
    dff.insert(0, "Region", loc[0])
    dff.insert(1, "Province", loc[1])
    dff.insert(2, "City", loc[2])
    dff.insert(3, "Barangay", loc[3])
    dff.insert(4, "Precinct ID", loc[4])
    df_container.append(dff)
    time.sleep(3)
    driver.back()
driver.quit()
df_final = pd.concat(df_container)
df_final

,Region,Province,City,Barangay,Precinct ID,"ABELLA, ERNIE (IND)","DE GUZMAN, LEODY (PLM)","DOMAGOSO, ISKO MORENO (AKSYON)","GONZALES, NORBERTO (PDSP)","LACSON, PING (PDR)","MANGONDATO, FAISAL (KTPNAN)","MARCOS, BONGBONG (PFP)","MONTEMAYOR, JOSE JR. (DPP)","PACQUIAO, MANNY PACMAN(PROMDI)","ROBREDO, LENI (IND)"
1,REGION III,AURORA,BALER,BARANGAY V (POB.),77010001,0,1,20,1,19,1,234,0,13,201
1,REGION III,AURORA,MARIA AURORA,BAZAL,77070017,0,0,5,1,4,0,320,0,8,42
